# Curso básico intensivo de JuMP (para MA4702)
## Prof. José Soto

***


# Gurobi y JuMP (Julia for Mathematical Programming)

Gurobi es un solver de optimización comercial, incluye:

- Linear programming (LP)
- Quadratic programming (QP)
- Quadratically constrained programming (QCP)
- Mixed integer linear programming (MILP)
- Mixed-integer quadratic programming (MIQP)
- Mixed-integer quadratically constrained programming (MIQCP).
- Non-Convex Quadratic Optimization (NCQO)

Hay varias formas de programar un modelo de optimización usando Gurobi en Julia, ejemplos en:
- https://github.com/JuliaOpt/Gurobi.jl

Se recomienda usar Gurobi con JuMP, <a hred="http://www.juliaopt.org/JuMP.jl/stable/">JuMP</a> es un lenguage de optimización matemática que soporta un gran abanico de solvers.

In [ ]:
# Para instalar o actualizar paquetes (si lo desea)  debe importar Pkg.
import Pkg
# Pkg.add("Gurobi")
# Pkg.add("JuMP")

In [ ]:
# Agreguemos paquetes para este tutorial

Pkg.add("GLPK") #otro solver gratuito
using Gurobi,JuMP, GLPK

** NOTA: Puede encontrar más información en https://jump.dev/JuMP.jl/v0.21.1/solutions/ y https://jump.dev/JuMP.jl/stable/quickstart/

# 1. Resolver PL simples (y PLM)

\begin{equation}
\begin{matrix}
\underset{x,y}{\max} & x+y \\
\textrm{s.t.} & 50x+21y & \leq & 2400 \\
& 30x+33y & \leq & 2100\\
& x & \geq & 5\\
& y & \geq & 45\\
\end{matrix}
\end{equation}

Presolve: operaciones que realiza un solver para transformar un problema de optimización en uno equivalente más fácil de resolver.

In [ ]:
#Creamos el objeto modelo usando una funcion
function creaModelo()
    ### MODELO, SOLVER y ATRIBUTOS
    # Indicar que el solver a utilizar es Gurobi
    # Cuando usemos varios modelos, lo mejor es pasarle la sesion de Gurobi
    #
    # nombremodelo = Model(Gurobi.Optimizer(GUROBI_ENV))
    # set_optimizer_attribute(nombremodelo, "nombreatributo1", valoratributo1)
    # ...
    
    mimodelo=Model(Gurobi.Optimizer(GUROBI_ENV))
    set_optimizer_attribute(mimodelo, "presolve", 1)
    
    ### VARIABLES
    # @variable(nombremodelo, variable declarada)
    # o bien
    # @variables(nombremodelo, begin
    #    var1
    #    ..
    #    varn
    # end)
    
    @variables(..
            
    ### Restricciones 
    # @constraint(nombremodelo, [(opcional) nombrerestriccion], restriccion)
    # o bien
    # @constraints(nombremodelo, begin
    # [nombrerestriccion], restriccion
    # ejemplos: 
    # rest1, a+b<=10
    # sum[b[i] for i in 1:n]<=10
    # salida[1:n], a[i]+b>=0
    # llamemosla const1 y const2
    @constraint(..
    @constraint(..
            
    ### Objetivo
    # @objective(nombremodelo, direccion, funcion)
    
    @objective(..

    return ..
end

In [ ]:
modelo1=creaModelo()

In [ ]:
optimize!(modelo1)

In [ ]:
#acceder a una variable del modelo (ojo, no es su valor)
modelo1[:x]

In [ ]:
#acceder a una restricción del modelo
modelo1[:const1]

In [ ]:
#preguntar por qué el solver paró
termination_status(modelo1)

In [ ]:
#Guardar el valor óptimo, la solución primal y los lados izquierdos de las restricciones.
obj= 
valorx= 
valory= 
LIconst1 = 
LIconst2 = 

@show(obj, valorx, valory, LIconst1, LIconst2)

In [ ]:
# Es posible también obtener inmediatamente la solución dual 
# En el dual hay una variable para cada restriccion (comando dual)
dual1=
dual2=

@show(dual1, dual2)


In [ ]:
#tambien hay una variable dual para cada cota, esto es mas dificil (manual) de obtener.
#
dualx=dual(LowerBoundRef(modelo1[:x]))
dualy=dual(LowerBoundRef(modelo1[:y]))
@show(dualx, dualy);

In [ ]:
#veamos que pasa si obligamos al que problema primal sea no acotado removiendo restricciones
modelo1=creaModelo()
delete(modelo1, modelo1[:const1])
delete(modelo1, modelo1[:const2])
optimize!(modelo1)

In [ ]:
termination_status(modelo1)

Resolvamos ahora un nuevo modelo para el siguiente problema entero

\begin{equation}
\begin{matrix}
\underset{x,y}{\max} & x+y \\
\textrm{s.t.} & 50x+21y & \leq & 2400 \\
& 30x+33y & \leq & 2100\\
& x & \geq & 5\\
& y & \geq & 45\\
&x, y &\in &\mathbb{Z}
\end{matrix}
\end{equation}

In [ ]:
modelo2=creaModelo()
set_integer(modelo2[:x])
set_integer(modelo2[:y])
modelo2

In [ ]:
optimize!(modelo2)

In [ ]:
#Guardar el valor óptimo, la solución primal y los lados izquierdos de las restricciones.
# ..
# @show(obj, valorx, valory, LIconst1, LIconst2)

# 2. Un modelo de transporte

Modelemos el clasico modelo de transporte siguiente para cierto grafo bipartito con Origen y Destino establecido. Aqui P y Q son vectores de demanda en ambos lados, y E es un conjunto de arcos en ORIGEN x DESTINO. Este es un programa lineal puro.

$$\begin{align*}
           \qquad \min \sum_{(a,b)\in E} c(a,b) x(a,b) \\
   \quad  \sum_{b: (a,b)\in E} x(a,b) &= P(a), \quad  \forall a \in \texttt{ORIG} \\
				      \sum_{a: (a,b)\in E} x(a,b) &=Q(b), \quad  \forall b \in \texttt{DEST} \\                           
x(a,b)&\geq 0,\quad \forall (a,b)\in E.
\end{align*}$$

Este modelo es válido solo cuando la suma de los P es igual a la suma de los Q.

La idea es modelar el problema pensando que E, c, P, Q son datos que serán revelados más tarde.
Pensamos que P y Q vienen como vectores y que c es la matriz que tiene sus valores (solo en E)

In [ ]:
import Test
function transporte(E, c, P, Q)
    mi_modelo = Model()

    #Le indicamos a JuMP que el solver a utilizar es Gurobi
    mi_modelo = Model(Gurobi.Optimizer(GUROBI_ENV))
    
    @variable(..
    @objective(..
    @constraints(mi_modelo, begin                    
        ..
    end)

    Test.@test sum(P) == sum(Q)
                    
    return mi_modelo
end
    

Probemos a resolver el problema para los siguientes valores

Valores1.

| P,Q,c  | 350  | 300 | 450 |
|--------|------|-----|-----|
|  400   | 30   | 45  | 10  |
|  400   | 10   | X   | 20  |
|  64    | 12   | 15  | 15  |
|  236   | 20   | 10  | X   |

Valores2.

| P,Q,c  | 30   | 20  | 10  | 70  |
|--------|------|-----|-----|-----|
|  20    | 5    | X   | 10  | 20  |
|  10    | X    | 80  | X   | 10  |
|  55    | X    | X   | X   | 20  |
|  10    | 10   | X   | 100 | X   |
|  5     | 20   | X   | X   | X   |
|  30    | 16   | 20  | X   | X   |



In [ ]:
modelo3=transporte(E,c,P,Q);

In [ ]:
optimize!(modelo3)

In [ ]:
objective_value(modelo3)
value.(modelo3[:x])

# 3. Modelar y resolver Caminos de Largo Minimo


 Estos son los dos modelos vistos en clases
 
 $$\begin{align*}
(\text{SP-Conector})\qquad\qquad \min &\sum_{e\in E}\ell_{e}x_{e}\\
x(\delta^+(U))&\geq 1 \text{ para todo $s$-$t$ corte $U$}\\
x&\in \{0,1\}^E
    \end{align*}$$

y

$$\begin{align*}
(\text{SP-Flujo})\qquad\qquad \min &\sum_{e\in E}\ell_{e}x_{e}\\
x(\delta^+(v))-x(\delta^-(v))&= 0 \text{ para todo $v\in V\setminus\{s,t\}$}\\
x(\delta^+(s))&=1\\
x(\delta^+(t))&=0\\
x(\delta^-(s))&=0\\
x(\delta^-(t))&=0\\
x&\in \{0,1\}^E
\end{align*}$$




In [ ]:
# Usaremos como instancia un grafo completo elegido al azar

N=8
largos = rand(1:100,N,N)
for i in 1:N
    largos[i,i]=0
end
origen = 2
destino = 7;
largos

In [ ]:
# Nota. para iterar sobre conjuntos se puede hacer
# Using combinatorics
#import Pkg
#Pkg.add("Combinatorics")
#using Combinatorics
for X in powerset([2 3 4 7])
    println(X)
end

In [ ]:
# Crear funcion para modelo SP-corte
function SP-corte(N, largos, origen, destino)
    ..
    return modeloSP-corte
end

# Crear funcion para modelo SP-flujo
function SP-flujo(N, largos, origen, destino)
    ..
    return modeloSP-flujo
end


modeloSP1=SP-corte(N,largos,origen,destino)
modeloSP2=SP-flujo(N,largos,origen,destino)
optimize!(modeloSP1)
objective_value(modeloSP1)
values.(modeloSP1[:x])

optimize!(modeloSP2)
objective_value(modeloSP2)
values.(modeloSP2[:x])
